<h1 style="color: blue">HD Wallet</h1>

## TOC:
* [Mnemonic Seed](#mneumonic-seed)
    * [Entropy](#entropy)
    * [Mnemonic creation](#mnemonic-create)
    * [Seeding](#seeding)
    * [Validation](#validation)
        * [Mnemonic validation](#mnemonic-validation)
        * [Seed validation](#seed-validation)
* [Extended Keys](#extended-keys)
    * [Master Extended Keys](#master-extended-keys)
    * [Key derivation](#key-derivation)
        * [Normal child extended private key](#normal-child-private)
        * [Hardened child extended private key](#hardened-child-private)
        * [Normal child extended public key](#normal-child-public)
        * [Hardened child extended public key](#hardened-child-public)
    * [Key derivation paths](#derivation-paths)
    * [Two parties key derivation test](#two-parties)
* [Encodings](#encodings)
    * [Extended key serialization](#extended-serialization)
    * [WIF](#wif)
    * [Addresses](#addreses)
* [Our HD-Wallet](#our-wallet)
    

In [62]:
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, PrivateFormat
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat import primitives


from ecpy.curves import Curve, Point # for elliptic curve math
from bitarray import bitarray, util 
from secrets import token_bytes # secure random source

import hashlib # mainly for RMD160 hash
import base58 # base 58 encoding

# Mneumonic seed <a class="anchor" id="mneumonic-seed"></a>

## Entropy <a clas="anchor" id="entropy"></a>
Generate a blob between 128 and 256 bits from a secure random source.
The length of the entropy should be multiple of 32.

In [63]:
def entropy_gen(n = 128):
    if n < 128 or n > 256:
        raise Exception("Entropy size must be between 128 and 256 bits inclusive.")
    if n % 32 != 0:
        raise Exception("Entropy size must be multiple of 32.")
    return token_bytes(n // 8)

entropy = entropy_gen(32*8)
print(f"Entropy (hex): {entropy.hex()}") #256 bits

Entropy (hex): 023bd671c9ba2345cb006c1262b79dc48086a92441ad1314543d888d20739bd9


## Mnemonic creation <a class="anchor" id="mnemonic-create"></a>

To make the entropy more human friendly we conver it to a series or words and introduce some error checking. To do so in this case we are gonna make use of __[BIP39-english](https://raw.githubusercontent.com/otromimi/bitcoin_testpad/wallet/BIP-0039_english.txt)__.

<div style="color:orange">
<h3>Atention</h3>
Mnemonic phrases are suported in many lenguages and alphabets. This notebook only uses the english ASCII variant.</br>
For other alphabets use UTF-8 NFKD encoding.
</div>

In [64]:
colors = {'red':'\033[91m', 'green':'\033[92m', 'yellow':'\033[93m', 'blue':'\033[94m', 'purple':'\033[95m', 'cian':'\033[96m', 'gray':'\033[90m'}

def mnemonic(bits):
    # Entorpy 2 binary
    entropy_bits = bitarray()
    entropy_bits.frombytes(bits)
    
    # Checksum
    hash256 = hashes.Hash(hashes.SHA256())
    hash256.update(bits)
    bin_h = bitarray()
    bin_h.frombytes(hash256.finalize())
    
    entropy_bits = entropy_bits + bin_h[:(len(entropy_bits) // 32)]

    numbers = [] # words numbers
    for i in range(len(entropy_bits)//11):
        numbers.append(util.ba2int(entropy_bits[11*i:11*i+11]))

    words = {} # words in our entropy
    with open('BIP-0039_english.txt', 'r') as file:
        for i, word in enumerate(file):
            if i in numbers:
                words[i] = word
                if len(numbers) == len(words):
                    break
    
    mnemonic_sentence = [] # mneumonic
    for i in numbers:
        mnemonic_sentence.append(words[i].strip()) 
    
    return tuple(mnemonic_sentence)


# change this assigment to check for different values.
entropy = entropy # Ex: entropy = entropy_gen()

mnemonic_sentence = " ".join(mnemonic(entropy))

# Entropy binary
entropy_bits = bitarray()
entropy_bits.frombytes(entropy)

# Entropy hash
hash256 = hashes.Hash(hashes.SHA256())
hash256.update(entropy)
bin_h = bitarray()
entropy_hash = hash256.finalize()
bin_h.frombytes(entropy_hash)

# Printing
if True:
    print(colors['purple']+entropy_bits.to01(), end="")
    print(colors['yellow']+bin_h.to01()[:(len(entropy_bits) // 32)], end="\n")
    print(f"{colors['purple']}\u2589 {'Entropy'}")
    print(f"{colors['yellow']}\u2589 {'Checksum (hash)'}\033[0m\n")

    print(f"Entropy (hex): {entropy.hex()}")
    print(f"entropy SHA-256: {entropy_hash.hex()}")

    print("\n\u250F"+"\u2501"*(len(mnemonic_sentence)+2)+"\u2513")
    print("\u2503 "+mnemonic_sentence+" \u2503")
    print("\u2517"+"\u2501"*(len(mnemonic_sentence)+2)+"\u251B")

del entropy_bits, hash256, bin_h, entropy_hash


000000100011101111010110011100011100100110111010001000110100010111001011000000000110110000010010011000101011011110011101110001001000000010000110101010010010010001000001101011010001001100010100010101000011110110001000100011010010000001110011100110111101100101010100
▉ Entropy
▉ Checksum (hash)

Entropy (hex): 023bd671c9ba2345cb006c1262b79dc48086a92441ad1314543d888d20739bd9
entropy SHA-256: 54984e472acd282bc36be4a7124937df6695fd27d1489475c76d2c1cd1e575fe

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ acquire team organ nature peasant pepper clown assault bar between trash math anchor fee much cup basic pencil author baby spirit degree hunt clever ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


## Seeding<a class="anchor" id="seeding"></a>
Derivates the seed that will be use for creating our cryptographic keys. We achive this by using an algorithm computer intensive that will slow down an attack.

Algorithm used: PBKDF2_HMAC (Password Based Key Derivation Function 2)<br>
* Hashing function: SHA-512 HMAC<br>
* Iterations: 2048<br>
* length: resulting key (seed) lenght in bytes<br>

Thanks to the use of a HMAC (Hash Message Authentication Code) hash function we can add a salt (password) to our nmonic phrase.



In [65]:
def mnemonic2seed(mnemonic_sentence, salt=""):
    
    mnemonic_dev = PBKDF2HMAC(
        algorithm = hashes.SHA512(),
        length = 64, #bytes
        salt = ("mnemonic"+salt).encode('ascii'),
        iterations = 2048
    )

    mnemonic_bytes = mnemonic_sentence.encode('ascii')
    seed = mnemonic_dev.derive(mnemonic_bytes)

    return seed


recovery_pass = "" # In case we want to add a salt to our mneumonic, add in here

seed = mnemonic2seed(mnemonic_sentence, recovery_pass)

if recovery_pass:
    print(f"Recovery password: {colors['red']}{recovery_pass}\033[0m")
print(f"Seed: {seed.hex()}")


Seed: 4c76c18e0ca7d0f4c5e14e0ed1f26ec3fa3797e8ed9959138bef362a4f2dcd6e591b886a5f819327e91d026b5debc5542d5c63b74a7566282f07bb535df4b2cc


## Validation<a class="anchor" id="validation"></a>

### Mnemonic validation<a class="anchor" id="mnemonic-validation"></a>
Validates de entropy against its hash to determinate if the mnemonic secuence was correct.

Requires: __[BIP39-english](https://raw.githubusercontent.com/otromimi/bitcoin_testpad/wallet/BIP-0039_english.txt)__.

In [66]:
def mnemonic_check(mnemonic):

    words = mnemonic.split(" ")
    numbers = [None] * len(words)

    with open('BIP-0039_english.txt') as file:
        for i, word in enumerate(file):
            if word.strip() in words:
                for j, item in enumerate(words):
                    if item == word.strip():
                        numbers[j] = i
    
    entropy = "".join([f'{i:011b}' for i in numbers])
    entropy = bitarray(entropy)

    checksum_len = len(entropy) % 32

    hash256 = hashes.Hash(hashes.SHA256())
    hash256.update(entropy[:-checksum_len].tobytes())
    bin_h = bitarray()
    entropy_hash = hash256.finalize()
    bin_h.frombytes(entropy_hash)
    
    if entropy[-checksum_len:] != bin_h[:checksum_len]:
        # Just raised in case checksum for mneumonic fails.
        raise Exception(f"Checksum fail, hash: {entropy_hash.hex()}")
    
    return entropy[:-checksum_len].tobytes()



recover_entropy = mnemonic_check(mnemonic_sentence)

print(f"{entropy.hex()} -> entropy") # entropy generated on first cell
print(f"{recover_entropy.hex()} -> recovered entropy")

del recover_entropy


023bd671c9ba2345cb006c1262b79dc48086a92441ad1314543d888d20739bd9 -> entropy
023bd671c9ba2345cb006c1262b79dc48086a92441ad1314543d888d20739bd9 -> recovered entropy


### Seed validation<a class="anchor" id="seed-validation"></a>

Validates the seed against the entropy and password.

In [67]:
def seed_check(seed, password):

    seed_chk = PBKDF2HMAC(
        algorithm = hashes.SHA512(),
        length = 64,
        salt = ("mnemonic"+password).encode('ascii'),
        iterations = 2048
    )
    seed_chk.verify(mnemonic_sentence.encode('ascii'), seed) # in case validation fails will raise a exception

if not seed_check(seed, recovery_pass):
    print("Seed check passed. \u2705")

Seed check passed. ✅


***

# Extended keys <a class="anchor" id="extended-keys"></a>

* [Master Extended Keys](#master-extended-keys)
* [Key derivation](#key-derivation)
    * [Normal child extended private key](#normal-child-private)
    * [Hardened child extended private key](#hardened-child-private)
    * [Normal child extended public key](#normal-child-public)
    * [Hardened child extended public key](#hardened-child-public)
* [Key derivation paths](#derivation-paths)
* [Two parties key derivation test](#two-parties)

## Master Extended Keys <a class="anchor" id="master-extended-keys"></a>


In [68]:
def master_key_extended(seed):

    expression = "Bitcoin seed" # Used for master key

    hmac_master = hmac.HMAC(expression.encode('ascii'), hashes.SHA512())
    hmac_master.update(seed)
    output = hmac_master.finalize()

    secret_num = output[:32]
    chain_code = output[32:]

    return ('m', secret_num, chain_code)


_, prv_master_key, chain_master_code = master_key_extended(seed)

private_key = ec.derive_private_key(int.from_bytes(prv_master_key, 'big'), ec.SECP256K1())
public_key = private_key.public_key()
pub_master_key = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)


print(f"Chain code: {chain_master_code.hex()}")
print(f"Private master key (d): {prv_master_key.hex()}")
print(f"Public master key (compressed): {pub_master_key.hex()}")

del private_key, public_key

Chain code: cb0dd20974e75c29ecf70dbcb344bb4d71113136d657b5985ff70db6cd0573c2
Private master key (d): 8d44363de1524829676c16dfd1b6de77f457d9e6b585f913e0eb03e4728d1cc2
Public master key (compressed): 02caa8865bd646ec95b42cc586c25e513c6104602f1d8578d72971b3e7106aad0a



## Key derivation<a class="anchor" id="key-derivation"></a>

1. [Normal child extended private key](#normal-child-private)
2. [Hardened child extended private key](#hardened-child-private)
3. [Normal child extended public key](#normal-child-public)
4. [Hardened child extended public key](#hardened-child-public)

In [69]:
def key_derivation(index, parent_key, parent_chain_code):

    index_bytes = index.to_bytes(32 // 8, 'big')
    data = parent_key + index_bytes
    salt = parent_chain_code

    hmac_512 = hmac.HMAC(salt, hashes.SHA512())
    hmac_512.update(data)
    output = hmac_512.finalize()

    return (output[:32], output[32:])

### Normal child extended private key<a class="anchor" id="normal-child-private"></a>

In [70]:
def private_normal_child(index, parent_prv_key, parent_chain_code):
    cv = Curve.get_curve('secp256k1')

    # Order of SECP256K1
    n = cv.order

    # Index range checking
    if index > 2147483647 | index < 0:
        raise f"Index: {index}, out of range for soft derivation."
    
    # Generating public key
    private_key = ec.derive_private_key(int.from_bytes(parent_prv_key, 'big'), ec.SECP256K1())
    public_key = private_key.public_key()
    parent_pub_key = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)

    # Derivation function
    child_prv_key, child_chain_code = key_derivation(index, parent_pub_key, parent_chain_code)

    # Checking child_chain_code
    if int.from_bytes(child_chain_code, 'big') > n:
        raise f"Chain code bigger than curve order \"n\"; Try with index {index + 1}"
    
    # Calculating child private key
    child_prv_key = ((int.from_bytes(child_prv_key, 'big') + int.from_bytes(parent_prv_key, 'big')) % n).to_bytes(32, 'big')

    return (index, child_prv_key, child_chain_code)




index, key, chain_code = private_normal_child(0, prv_master_key, chain_master_code)

private_key = ec.derive_private_key(int.from_bytes(key, 'big'), ec.SECP256K1())
public_key = private_key.public_key()
public_compress_child = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)

print(f"Index: {index}")
print(f"Chain code: {chain_code.hex()}")
print(f"Private key (d): {key.hex()}")
print(f"Public key (compressed): {public_compress_child.hex()}")

del index, key, chain_code, private_key, public_key, public_compress_child

Index: 0
Chain code: 886ae1366e80b90c082f65a742054d32c4a87730b1bb3863f41bab615b632179
Private key (d): 0cbd073975dfcb86b65f7580332aea2882126e8617c37a5104d919ce63bc1db0
Public key (compressed): 03a1ddc497e70dbb260c9ec44a6a2f416f17a3c7ddcd63d35ac0f30f6ddc59ab71


### Hardened child extended private key<a class="anchor" id="hardened-child-private"></a>

In [71]:
def private_hardened_child(index, parent_prv_key, parent_chain_code):
    cv = Curve.get_curve('secp256k1')

    # Order of SECP256K1
    n = cv.order

    # Index range checking
    if index > 2147483647 | index < 0:
        raise f"Index: {index}, Hard index: {index + 2147483648}, out of range for hard derivation."
    
    # Generating public key --> No required

    # Derivation function
    child_prv_key, child_chain_code = key_derivation(
        index + 2147483648, # Hard index range
        b'\x00' + parent_prv_key, # Leading bit for public key compensation
        parent_chain_code
    )

    # Checking child_chain_code
    if int.from_bytes(child_chain_code, 'big') > n:
        raise f"Chain code bigger than curve order \"n\"; Try with index {index + 1}"
    
    # Calculating child private key
    child_prv_key = ((int.from_bytes(child_prv_key, 'big') + int.from_bytes(parent_prv_key, 'big')) % n).to_bytes(32, 'big')

    return (index, child_prv_key, child_chain_code)




index, key, chain_code = private_hardened_child(0, prv_master_key, chain_master_code)

private_key = ec.derive_private_key(int.from_bytes(key, 'big'), ec.SECP256K1())
public_key = private_key.public_key()
public_compress_child = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)


print(f"Index: {index}, Hard index: {index + 2147483648}")
print(f"Chain code: {chain_code.hex()}")
print(f"Private key (d): {key.hex()}")
print(f"Public key (compressed): {public_compress_child.hex()}")

del index, key, chain_code, private_key, public_key, public_compress_child

Index: 0, Hard index: 2147483648
Chain code: 6c9a32d7bf6da60220d4a763032bd7b483ae41e1ba86b83406280a34a1720831
Private key (d): 0e4605b97bb28415fc6732d94f41d55fa1bc134e5a4045cea95813b0b0a1fdf2
Public key (compressed): 028e311f4c08969a1ac09f9e99d4a1925e1a4010707d41b1adeee9d4b65dc169da


### Normal child extended public key<a class="anchor" id="normal-child-public"></a>

In [72]:
def public_normal_child(index, parent_pub_key, parent_chain_code):
    cv = Curve.get_curve('secp256k1')

    # Order of SECP256K1
    n = cv.order

    # Index range checking
    if index > 2147483647 | index < 0:
        raise f"Index: {index}, out of range for soft derivation."
    
    # Generating public key --> No required, we pass the pub key as parameter

    # Derivation function
    child_pub_key, child_chain_code = key_derivation(index, parent_pub_key, parent_chain_code)

    # Checking child_chain_code
    if int.from_bytes(child_chain_code, 'big') > n:
        raise f"Chain code bigger than curve order \"n\"; Try with index {index + 1}"

    # Calculating child private key
    parent_point = cv.decode_point(parent_pub_key)
    child_point = cv.mul_point(int.from_bytes(child_pub_key, 'big'), cv.generator)
    result_point = parent_point + child_point

    # Encoding in weird format because ECPy doesn't support compress point encoding
    child_pub_key = ec.EllipticCurvePublicKey.from_encoded_point(
        ec.SECP256K1(), 
        bytes(cv.encode_point(result_point))).public_bytes(
            Encoding.X962, 
            PublicFormat.CompressedPoint
        )
    
    return (index, child_pub_key, child_chain_code)




index, key, chain_code = public_normal_child(0, pub_master_key, chain_master_code)

print(f"Index: {index}")
print(f"Chain code: {chain_code.hex()}")
print(f"Parent public key (compressed): {pub_master_key.hex()}")
print(f"Public key (compressed): {key.hex()}")

del index, key, chain_code

Index: 0
Chain code: 886ae1366e80b90c082f65a742054d32c4a87730b1bb3863f41bab615b632179
Parent public key (compressed): 02caa8865bd646ec95b42cc586c25e513c6104602f1d8578d72971b3e7106aad0a
Public key (compressed): 03a1ddc497e70dbb260c9ec44a6a2f416f17a3c7ddcd63d35ac0f30f6ddc59ab71


### Hardened child extended public key<a class="anchor" id="hardened-child-public"></a>

In [73]:
def public_hardened_child(index, parent_pub_key, parent_chain_code):
    raise f"You need the private key to derive its peer public key."


## Key derivation paths <a class="anchor" id="derivation-paths"></a>

In [74]:
def derive_path(path, seed):
    path_list = path.split('/')
    index, key, chain_code = None, None, None
    for term in path_list:
        if term == 'm':
            # Master extended key from seed
            _, key, chain_code = master_key_extended(seed)
            continue
        if "'" in term:
            # Case term is hard
            _, key, chain_code = private_hardened_child(int(term.strip("'")), key, chain_code)
        else:
            # Case the term is soft of normal
            _, key, chain_code = private_normal_child(int(term), key, chain_code)
    return key, chain_code

derivation_path = "m/44'/0'/0'/0/0" # Path extipulated on BIP 44

path_key, _ = derive_path(derivation_path, seed)

print(f"Private key on {derivation_path}:\n\t{path_key.hex()}")

del derivation_path, path_key

Private key on m/44'/0'/0'/0/0:
	87120a8b13f32feb427bec79973d97e67a8ce75cef395e317c672a40076d86a7


## Two parties key derivation test <a class="anchor" id="two-parties"></a>

For this test we are gonna derive in parallel the public and private key; and then sign and verify a message with those keys independently derived.

In [75]:
# Finding the master pub, prv and chain-code
prv_secret, chain_code_m = derive_path("m", seed)
prv_key_m = ec.derive_private_key(int.from_bytes(prv_secret, 'big'), ec.SECP256K1())
pub_key_m = prv_key_m.public_key().public_bytes(Encoding.X962, PublicFormat.CompressedPoint)

# Derive our path for our private key
prv_secret, _ = derive_path("m/444/8", seed) # paths must be the same
prv_key = ec.derive_private_key(int.from_bytes(prv_secret, 'big'), ec.SECP256K1())

# We derive the same path for our public key
_, pub_key, chain_code_child = public_normal_child(444, pub_key_m, chain_code_m) # 444/
_, pub_key, chain_code_child = public_normal_child(8, pub_key, chain_code_child) # 8/
pub_key = ec.EllipticCurvePublicKey.from_encoded_point(ec.SECP256K1(), pub_key)

# We sign a message with our derived private key
message = b"Testing derivaiton of extended keys."
signature = prv_key.sign(message, ec.ECDSA(hashes.SHA256()))

# If no exception raised then the signature is correct
pub_key.verify(signature, message, ec.ECDSA(hashes.SHA256()))


del prv_secret, prv_key_m, pub_key_m, prv_key, pub_key, message, signature, \
    chain_code_child, chain_code_m

***

# Encodings <a class="anchor" id="encodings"></a>

* [Extended key serialization](#extended-serialization)
* [WIF](#wif)
* [Addresses](#addreses)

## Extended key serialization <a class="anchor" id="extended-serialization"></a>

version + depth + fingerprint + childnumber + chain_code + key + (checksum)

* version:     puts xprv or xpub at the start
* depth:       how many times this child has been derived from master key (0 = master key)
* fingerprint: created from parent public key (allows you to spot adjacent xprv and xpubs)
* childnumber: the index of this child key from the parent
* chain_code:  the current chain code being used for this key
* key:         the private or public key you want to create a serialized extended key for (prepend 0x00 for private)

In [76]:
def extended_serialization(depth, parent_key, child_number, chain_code, key, version = None, testnet = True):

    # Version BIP 44
    if version == None and testnet == False:
        serialized_key = b"\x04\x88\xad\xe4" if len(key) == 32 else b"\x04\x88\xb2\x1e"
    elif version == None and testnet == True:
        serialized_key = b"\x04\x35\x83\x94" if len(key) == 32 else b"\x04\x35\x87\xcf"
    else:
        serialized_key = verison
   
    # Depth
    serialized_key += depth.to_bytes(1, 'big')

    # Fingerpirnt of the pub father key
    hash1 = hashlib.sha256(parent_key)
    hash2 = hashlib.new('rmd160')
    hash2.update(hash1.digest())
    fingerprint = hash2.digest()[:4]

    # Child index
    index = child_number.to_bytes(4, 'big')

    serialized_key += fingerprint + index + chain_code

    # If pvt key prepend 0x00
    if len(key) == 32:
        serialized_key += b"\x00"

    serialized_key += key

    # Checksum
    hash1 = hashlib.sha256(serialized_key)
    hash2 = hashlib.sha256(hash1.digest())

    serialized_key += hash2.digest()[:4]

    return base58.b58encode(serialized_key)


# Derivaiton paths
derivation_path = "m/0" #44'/0'/0'/0/0
parent_derivation_path = "/".join(derivation_path.split('/')[:-1])

# Generating necesary keys
prv_key, chain_code = derive_path(derivation_path, seed)
parent_prv_key, _ = derive_path(parent_derivation_path, seed)
parent_prv_key = ec.derive_private_key(int.from_bytes(parent_prv_key, 'big'), ec.SECP256K1())
parent_pub_key = parent_prv_key.public_key().public_bytes(Encoding.X962, PublicFormat.CompressedPoint)

# Getting index and depth
derivation_count = len(derivation_path.split('/'))-1
child_n = int(derivation_path.split('/')[-1].strip("'"))

# Serialization
serialized_key = extended_serialization(derivation_count, parent_pub_key, child_n, chain_code, prv_key)

# Printing
if True:
    print(f"Parent derivation path: {parent_derivation_path}")
    print(f"Parent public key: {parent_pub_key.hex()}\n")

    print(f"Derivation path: {derivation_path}")
    print(f"Key: {prv_key.hex()}")
    print(f"Chain code: {chain_code.hex()}\n")

    print(colors['red']+base58.b58decode(serialized_key).hex()[0:8], end="")
    print(colors['blue']+base58.b58decode(serialized_key).hex()[8:10], end="")
    print(colors['purple']+base58.b58decode(serialized_key).hex()[10:18], end="")
    print(colors['cian']+base58.b58decode(serialized_key).hex()[18:26], end="")
    print(colors['yellow']+base58.b58decode(serialized_key).hex()[26:90], end="")
    print(colors['green']+base58.b58decode(serialized_key).hex()[90:156], end="")
    print(colors['gray']+base58.b58decode(serialized_key).hex()[156:], end="\n")

    print(f"{colors['red']}\u2589 {'Version'}", end="\t")
    print(f"{colors['blue']}\u2589 {'Depth'}")
    print(f"{colors['purple']}\u2589 {'Fingerprint'}", end="\t")
    print(f"{colors['cian']}\u2589 {'Child index'}")
    print(f"{colors['yellow']}\u2589 {'Chain code'}", end="\t")
    print(f"{colors['green']}\u2589 {'Key'}")
    print(f"{colors['gray']}\u2589 {'Checksum'}", end="\033[0m\n")

    print("\n\u250F"+"\u2501"*(len(serialized_key.decode('ascii'))+2)+"\u2513")
    print("\u2503 "+serialized_key.decode('ascii')+" \u2503")
    print("\u2517"+"\u2501"*(len(serialized_key.decode('ascii'))+2)+"\u251B")

del prv_key, chain_code, parent_prv_key, parent_pub_key, derivation_path, \
    parent_derivation_path, derivation_count, child_n, serialized_key

Parent derivation path: m
Parent public key: 02caa8865bd646ec95b42cc586c25e513c6104602f1d8578d72971b3e7106aad0a

Derivation path: m/0
Key: 0cbd073975dfcb86b65f7580332aea2882126e8617c37a5104d919ce63bc1db0
Chain code: 886ae1366e80b90c082f65a742054d32c4a87730b1bb3863f41bab615b632179

04358394010168139700000000886ae1366e80b90c082f65a742054d32c4a87730b1bb3863f41bab615b632179000cbd073975dfcb86b65f7580332aea2882126e8617c37a5104d919ce63bc1db0067bc5e6
▉ Version	▉ Depth
▉ Fingerprint	▉ Child index
▉ Chain code	▉ Key
▉ Checksum

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ tprv8bagz7aPwGXBBn5ot9iJkphgTNkpZyrwyptN8cG5dk8Nko847U5tfUSFeje4sFc93YukLWvXenpL2Q1bePYz6sSkraX89VLtYktx1EESt2h ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


## WIF (wallet import format)<a class="anchor" id="wif"></a>

A simple way to move private keys arround that contains the following fields in a human friendly base 54 encoding.
1. Network (mainet or testnet)
2. Private key (d)
3. Compress byte (denotates if pub keys are compressed)
4. Checksum

In [77]:
def wif(key, compressed = True, testnet = True):

    # Network
    network = b"\xef" if testnet else b"\x80"
    
    # Network + key + compress byte
    result = network + key + (b"\x01" if compressed else b"")
    
    # Checksum
    hash1 = hashlib.sha256(result)
    hash2 = hashlib.sha256(hash1.digest())
    result += hash2.digest()[0:4]

    return base58.b58encode(result)


wif_key = wif(prv_master_key)

# Prints
if True:
    print(colors['red']+base58.b58decode(wif_key).hex()[0:2], end="")
    print(colors['blue']+base58.b58decode(wif_key).hex()[2:66], end="")
    if len(base58.b58decode(wif_key).hex()) > 37 * 2:
        print(colors['yellow']+base58.b58decode(wif_key).hex()[66:68], end="")
        print(colors['purple']+base58.b58decode(wif_key).hex()[68:], end="\n")
    else:
        print(colors['purple']+base58.b58decode(wif_key).hex()[66:], end="\n")

    print(f"{colors['red']}\u2589 {'Network'}")
    print(f"{colors['blue']}\u2589 {'Key'}")
    print(f"{colors['yellow']}\u2589 {'Compress byte'}")
    print(f"{colors['purple']}\u2589 {'Checksum'}", end="\033[0m\n")

    print("\n\u250F"+"\u2501"*(len(wif_key.decode('ascii'))+2)+"\u2513")
    print("\u2503 "+wif_key.decode('ascii')+" \u2503")
    print("\u2517"+"\u2501"*(len(wif_key.decode('ascii'))+2)+"\u251B")

del wif_key

ef8d44363de1524829676c16dfd1b6de77f457d9e6b585f913e0eb03e4728d1cc20182234a49
▉ Network
▉ Key
▉ Compress byte
▉ Checksum

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ cSKJepurrT4sLdDtF3YRRkXmYK1cpFwhZkNv7MdNwE3npyxC7zpY ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


## Addresses (P2PKH)<a class="anchor" id="addreses"></a>

Bitcoin addresses for P2PKH (Pay to public key hash).

In [78]:
def P2PKH(pub_key, testnet = True):
    # Network
    network = b"\x6f" if testnet else b"\x00"

    # hash160
    hash1 = hashlib.sha256(pub_key)
    hash2 = hashlib.new('rmd160')
    hash2.update(hash1.digest())
    result = network + hash2.digest()

    # Checksum
    hash1 = hashlib.sha256(result)
    hash2 = hashlib.sha256(hash1.digest())
    result += hash2.digest()[:4]

    return base58.b58encode(result)

address = P2PKH(pub_master_key, False)

# Prints
if True:
    print(colors['red']+base58.b58decode(address).hex()[0:2], end="")
    print(colors['blue']+base58.b58decode(address).hex()[2:-8], end="")
    print(colors['purple']+base58.b58decode(address).hex()[-8:], end="\n")

    print(f"{colors['red']}\u2589 {'Version'}")
    print(f"{colors['blue']}\u2589 {'hash160(pub_key)'}")
    print(f"{colors['purple']}\u2589 {'Checksum'}", end="\033[0m\n")

    print("\n\u250F"+"\u2501"*(len(address.decode('ascii'))+2)+"\u2513")
    print("\u2503 "+address.decode('ascii')+" \u2503")
    print("\u2517"+"\u2501"*(len(address.decode('ascii'))+2)+"\u251B")

del address

00016813978d10c2d54bdf8be41468cb5a0f551fb3a961a7c8
▉ Version
▉ hash160(pub_key)
▉ Checksum

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 18SMZFNHx45crCU2ddeLPZMhmXg2RHFGB ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


***
# Our HD-Wallet <a class="anchor" id="our-wallet"></a>

In [79]:

entropy = entropy_gen(256)
mnemonic_sentence = " ".join(mnemonic(entropy))

print("\n\u250F"+"\u2501"*(len(mnemonic_sentence)+2)+"\u2513")
print("\u2503 "+mnemonic_sentence+" \u2503")
print("\u2517"+"\u2501"*(len(mnemonic_sentence)+2)+"\u251B")

seed = mnemonic2seed(mnemonic_sentence)

def encoded_key_pairs(prv_key, chain_code, testnet = True):
    # WIF for private key
    wif_key = wif(prv_key, True, testnet)

    # Getting public key
    prv_key = ec.derive_private_key(int.from_bytes(prv_key, 'big'), ec.SECP256K1())
    pub_key = prv_key.public_key().public_bytes(Encoding.X962, PublicFormat.CompressedPoint)

    # Address from public key
    address = P2PKH(pub_key, testnet)
    
    return (wif_key, address)


derivation_path = "m/44'/1'/0'/0/0" # Path testnet BIP 44

if True:
    print("{:18}{:38}{:55}".format('Path'.center(18),'Address'.center(38),'WIF'.center(55)))
    print(colors['gray']+"-"*111, end='\033[0m\n')

    # Ordinary
    for i in range(10): # Wallets search on the first 20 addresses
        derivation = derivation_path.split('/')
        derivation[-1] = str(int(derivation_path[-1]) + i)

        extended_key = derive_path("/".join(derivation), seed)
        key_pair = encoded_key_pairs(*extended_key, True)

        print("{:>18}".format("/".join(derivation)), end="")
        print("{}{:>38}".format(colors['blue'], key_pair[1].decode('ascii')), end="")
        print("{}{:>55}".format(colors['red'], key_pair[0].decode('ascii')), end="\033[0m\n")

    # Change
    for i in range(5):
        derivation = derivation_path.split('/')
        derivation[-2] = '1'
        derivation[-1] = str(int(derivation_path[-1]) + i)

        extended_key = derive_path("/".join(derivation), seed)
        key_pair = encoded_key_pairs(*extended_key, True)

        print("{:>18}".format("/".join(derivation)), end="")
        print("{}{:>38}".format(colors['yellow'], key_pair[1].decode('ascii')), end="")
        print("{}{:>55}".format(colors['red'], key_pair[0].decode('ascii')), end="\033[0m\n")
    
del derivation_path, extended_key, key_pair, derivation



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ edge this govern chuckle garbage response survey alter love caution stool cram potato hip brief convince abuse hamster festival have extra hope buzz field ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
       Path                      Address                                          WIF                          
---------------------------------------------------------------------------------------------------------------
   m/44'/1'/0'/0/0    n21wiNHMC47MjHUtL5Xdbt37GPL324S79E   cSFVJAGeWxMckbvkarsbn9h7sk6bQvTea9h2vZUZuTSMXJoFQUGg
   m/44'/1'/0'/0/1    mq3Geva4k8fuUfqRm2geZ2unw3oQbr5qzF   cW2Pykf5qgRrfGZcH8zdC1f771TpCcwZDLTknnTAyLa2nW1WFqgp
   m/44'/1'/0'/0/2    muH3rbNnPCU6t8vPqL6ihhBozftkZJWoTy   cVE424SG3KhetkD